In [ ]:
# !pip3 install torch

In [ ]:
# !pip3 install tqdm

In [ ]:
# !pip3 install transformers

In [ ]:
import torch
torch.cuda.empty_cache()
torch.__version__

'1.9.1+cu102'

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('/userdirs/piyumal/roberta_sinhala/content/sinhala-dataset-creation/datasets/tokenized/').glob('**/*.txt')]

In [ ]:
len(paths)

158

In [ ]:
# from tokenizers import ByteLevelBPETokenizer

# tokenizer = ByteLevelBPETokenizer()

In [ ]:
# tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2,
#                 special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [ ]:
# import os

# os.mkdir('./Roberta_tokenizer')

# tokenizer.save_model('Roberta_tokenizer')
# tokenizer.save("./Roberta_tokenizer/config.json") 
# tokenizer.save("./Roberta_tokenizer/tokenizer.json") 

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('Roberta_tokenizer', max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [ ]:
tokens = tokenizer("එවිට දවසින් දවස අවතැන්වූවන් සිටින කඳවුරුවලට මෙම සෞඛ්‍ය කණ්ඩායම් යවනවා")

In [ ]:
print(tokens)

{'input_ids': [0, 1056, 265, 280, 599, 265, 266, 264, 599, 1702, 284, 266, 264, 267, 320, 329, 264, 286, 265, 280, 265, 266, 1366, 272, 270, 272, 574, 288, 276, 273, 286, 501, 490, 294, 269, 586, 264, 304, 268, 415, 264, 3476, 268, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
with open('Sinhala_all_data.txt', 'r', encoding='utf-8') as fp:
    lines = fp.read().split('\n')

In [ ]:
lines[:100]

['',
 'ලාල් රූපතුංග රතිඥාවක් පත්තු කිරීමට උත්සහ දරන විට ලියනපතිරණ රතිඤාවක් පත්තු කොට පහත මාලයට විසි කලේය',
 'ජනතාව ජන්දයෙන් ප්\u200dරතික්ෂේප කළ අය ද දැන් ඒ ආණ්ඩුවේ නිලතල දරති',
 'ජයවිලාල් විලේගොඩගේ නොපළ ලිපි සුනිල් මිහිඳුකුල පි',
 'අධිරාජ්\u200dයයට ගැති වෙති',
 'ප්\u200dරශ්නෙකට කියල ලොකුවටම ආවෙ එයා පිට රට ඉන්න එයාගෙ යාලුවෙක් කියල කියපු ගර්ල් කෙනෙක් එක්ක ඕනෙවට වඩා කතාකරන්න පටං ගත්ත එක තමයි',
 'මමත් එහෙම වෙලා හිටිය නිසා මම අත්දැකීමෙන් දන්නවා ඒදේ',
 'ඔයාල කරල තියෙන එකේ එක හරියටම බලාගන්න නම් එකේ කියන තැන බලන්න',
 'නීත්\u200dයනුකූල ව රට යන්න',
 'එය ශ්\u200dරී ලංකාවාසීන්ට නැවුම් අත්දැකීමක් වන්නේ එය මුහුද ගොඩකර ඉදිකෙරුණු නගරයක් නිසාවෙනි',
 'ඒ පිළිබඳ නිවේදනද නිකුත් කරයි',
 'අප යුක්තිය පුවත්පත පවත්වාගෙන ගිය කාලයේ සිංහල බෞද්ධාගමේ මුදුන්මල්කඩ වන භික්ෂූන් පවා රහමෙරට ආසා කරන බවට නිශ්චිත සාක්ෂි තිබුණි',
 'හිටපු ජනාධිපති මහින්ද රාජපක්ෂ මහතාගේ පියා වූ ඩී.ඒ. රාජපක්ෂ මහතාගේ ගුණ සමරු සටහනක් සඳහා මෙසේ පාසල් සිසු සිසුවියන් ගෙන්වා තිබුණි',
 'විකට රංගන ශිල්පී පින් පොන් සමාවගත් වීඩියෝව ඉවත් කරගනී',
 'තරහා ගන්

In [ ]:
len(lines)

15790635

In [ ]:
def mlm(input_ids):
    # input_ids = labels.detach().clone()
    # create random array of floats with equal dims to input_ids
    rand = torch.rand(input_ids.shape)
    # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
    mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
    # loop through each row in input_ids tensor (cannot do in parallel)
    for i in range(input_ids.shape[0]):
        # get indices of mask positions from mask array
        # print("mask array shape:", mask_arr.shape)
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        # print("selection array shape:", selection)
        # mask input_ids
        input_ids[i, selection] = 4 
    return input_ids

In [ ]:
import torch
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [ ]:
# batch = tokenizer(lines[:100_000], max_length=512, padding='max_length', truncation=True)
# len(batch)

In [ ]:
# # make copy of labels tensor, this will be input_ids
# input_ids = labels.detach().clone()
# # create random array of floats with equal dims to input_ids
# rand = torch.rand(input_ids.shape)
# # mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
# mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
# # loop through each row in input_ids tensor (cannot do in parallel)
# for i in range(input_ids.shape[0]):
#     # get indices of mask positions from mask array
#     print("mask array shape:", mask_arr.shape)
#     selection = torch.flatten(mask_arr[i].nonzero()).tolist()
#     print("selection array shape:", selection)
#     # mask input_ids
#     input_ids[i, selection] = 4  # our custom [MASK] token == 3

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

In [ ]:
from tqdm.auto import tqdm
import gc
import pickle

epochs = 1
loss_values = []
for epoch in range(epochs):
    for i in range(0, len(lines), 5_00_000):
        if i == (len(lines)//5_00_000)*5_00_000:
            batch_data = tokenizer(lines[i:], max_length=512, padding='max_length', truncation=True)
        else:
            batch_data = tokenizer(lines[i:i+5_00_000], max_length=512, padding='max_length', truncation=True)
        labels_all = torch.tensor(batch_data.input_ids)
        mask_all = torch.tensor(batch_data.attention_mask)
        input_ids_all = mlm(labels_all.detach().clone())
        encodings = {'input_ids': input_ids_all, 'attention_mask': mask_all, 'labels': labels_all}
        dataset = Dataset(encodings)
        loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
    # setup loop with TQDM and dataloader
        loop = tqdm(loader, leave=True)
        for batch in loop:
            # initialize calculated gradients (from prev step)
            optim.zero_grad()
            # pull all tensor batches required for training
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            # process
            outputs = model(input_ids, attention_mask=attention_mask,
                            labels=labels)
            # extract loss
            loss = outputs.loss
            # calculate loss for every parameter that needs grad update
            loss.backward()
            # update parameters
            optim.step()
            # print relevant info to progress bar
            loop.set_description(f'Epoch: {epoch} Text Loading Iter: {i//5_00_000}')
            loop.set_postfix(loss=loss.item())
            loss_values.append(loss.item())
            del input_ids
            del attention_mask
            del labels
        else:
            model.save_pretrained(f'./Roberta_models/roberta_model_{epoch}_{i}')
            del batch_data
            del labels_all 
            del mask_all 
            del input_ids_all
            gc.collect()
            with open('loss_value_file', 'wb') as fp:
                pickle.dump(loss_values, fp)

  0%|          | 0/31250 [00:00<?, ?it/s]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [ ]:
# model.save_pretrained('./Test_on_the_way_bert')

In [ ]:
from transformers import pipeline
fill = pipeline('fill-mask', model='Test_on_the_way_bert', tokenizer=tokenizer)

In [ ]:
fill(f'මමත් එහෙම වෙලා හිටිය නිසා {fill.tokenizer.mask_token} අත්දැකීමෙන් දන්නවා ඒද?')